In [1]:
from IPython.display import clear_output
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import re
import dropbox
import sys
import pandas as pd
import numpy as np
import imageio.v2 as imageio
import matplotlib.pyplot as plt
import cv2
import logging
import datetime
import glob
import json
import scipy
import matplotlib as mpl

from subprocess import call
from tifffile import imwrite
from tqdm import tqdm
from pathlib import Path
from amftrack.util.dbx import (
    upload_folder,
    download,
    read_saved_dropbox_state,
    save_dropbox_state,
    load_dbx,
    get_dropbox_folders_prince,
    get_dropbox_video_folders,
    download_video_folders_drop,
    download_analysis_folders_drop,
)
from amftrack.pipeline.launching.run import (
    run_transfer,
)
from amftrack.pipeline.launching.run_super import run_parallel_transfer
from amftrack.pipeline.launching.run_super import run_parallel_flows
from amftrack.pipeline.functions.transport_processing.high_mag_videos.plot_data import (
    plot_summary,
    save_raw_data,
)
from amftrack.pipeline.functions.transport_processing.high_mag_videos.high_mag_analysis import (
    HighmagDataset,
    VideoDataset,
    EdgeDataset,
    index_videos_dropbox_new,
    analysis_run,
)
from amftrack.pipeline.functions.transport_processing.high_mag_videos.kymo_class import (
    KymoVideoAnalysis,
    KymoEdgeAnalysis,
)
from amftrack.util.dbx import upload

%load_ext autoreload
%autoreload 2
%matplotlib widget

from amftrack.pipeline.launching.run_super import (
    run_launcher,
    directory_scratch,
    directory_project,
    directory_project,
    run_parallel_stitch,
    run_parallel_transfer,
)
import os
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Experiment,
    save_graphs,
    load_graphs,
    Edge,
    Node,
)
from amftrack.pipeline.functions.image_processing.experiment_util import (
    get_random_edge,
    distance_point_edge,
    plot_edge,
    plot_edge_cropped,
    find_nearest_edge,
    get_edge_from_node_labels,
    plot_full_image_with_features,
    get_all_edges,
    get_all_nodes,
    find_neighboring_edges,
    reconstruct_image,
    reconstruct_skeletton_from_edges,
    reconstruct_skeletton_unicolor,
    reconstruct_image_from_general,
    plot_full,
    plot_edge_color_value,
)
from amftrack.transport.align_video_network import identify_nodes

logging.basicConfig(stream=sys.stdout, level=logging.debug)
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    update_analysis_info,
    get_analysis_info,
    get_current_folders,
    get_folders_by_plate_id,
)

mpl.rcParams["figure.dpi"] = 100
from amftrack.pipeline.functions.transport_processing.high_mag_videos.register_videos import *
from amftrack.pipeline.functions.transport_processing.high_mag_videos.loading import (
    load_video_dataset,
)
from amftrack.pipeline.functions.transport_processing.high_mag_videos.make_ML_dataset import *

/gpfs/home2/cbisot/pycode/AMFtrack/amftrack/util/dbx.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
2024-06-26 09:41:12.632595: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-26 09:41:12.742220: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-26 09:41:13.025176: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-26 09:41:21.847961: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2024-06-26 09:41:37,572-[WARNING]- absl:256 -> Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
2024-06-26 09:41:37,585-[WARNING]- absl:184 -> Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.


/home/cbisot/miniconda3/envs/amftrack/lib/python3.12/site-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [2]:
import shutil

shutil.rmtree("/scratch-shared/amftrack/ML_dataset/", ignore_errors=True)
os.mkdir("/scratch-shared/amftrack/ML_dataset/")

In [3]:
plate_id = "310_20230830"
plate_id_video = "20230903_Plate310"
# plate_id = "441_20230807"

videos_folder = "/projects/0/einf914/videos/"

analysis_folder = "/projects/0/einf914/analysis_videos/CocoTransport/"
analysis_folder_root = "/projects/0/einf914/analysis_videos/"
directory_targ = "/projects/0/einf914/transport/"

In [4]:
indexes ={
        "20230901_Plate310": "20230901_0719_Plate06",
        "20230902_Plate310": "20230902_1343_Plate07",
        "20230903_Plate310": "20230903_1143_Plate07",
        "20230904_Plate310": "20230904_0942_Plate07",
        "20230905_Plate310": "20230905_1345_Plate07",
        # "20230906_Plate310" : "20230906_1220_Plate07",
    }
# indexes = {
# "20230810_Plate441" : "20230810_1005_Plate14",
# "20230811_Plate441" : "20230811_1605_Plate14",
# "20230812_Plate441" : "20230812_1006_Plate14",
# "20230813_Plate441" : "20230813_1618_Plate14",
# }

In [5]:
update_plate_info(directory_targ, local=True)
all_folders = get_current_folders(directory_targ, local=True)
folders = all_folders.loc[all_folders["unique_id"] == plate_id]
folders = folders.loc[folders["/Analysis/nx_graph_pruned_labeled.p"] == True]
folders = folders.sort_values(by="datetime")

analysed:   0%|          | 0/494 [00:00<?, ?it/s]

In [6]:
for plate_id_video in list(indexes.keys()):
    data_obj = load_video_dataset(
        plate_id_video, videos_folder, analysis_folder, analysis_folder_root
    )

    exp = Experiment(directory_targ)
    i = np.where(folders['folder'] == indexes[plate_id_video])[0][0]

    selection = folders.iloc[i:i+2]
    exp.load(selection, suffix="_labeled")
    for t in range(exp.ts):
        exp.load_tile_information(t)

    data_obj.video_objs = sorted(
        data_obj.video_objs, key=lambda video: video.dataset["video_int"]
    )
    make_images(data_obj, videos_folder)
    make_profile(data_obj, exp, 0)
    # break
# from amftrack.util.dbx import upload_folder

path = f"/scratch-shared/amftrack/ML_dataset/"
target_drop = "/DATA/CocoTransport/NewMLDataset"
upload_folder(path, target_drop, delete=False)

/gpfs/home2/cbisot/pycode/AMFtrack/amftrack/pipeline/functions/image_processing/experiment_class_surf.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.folders["datetime"] = pd.to_datetime(


2023-09-01 07:19:00
2023-09-01 13:43:00
(array([26999.0518066 , 36658.06064438]), array([26913.38562885, 36743.51417555]))
(array([26983.07045909, 36643.04077314]), array([26897.40428135, 36728.49430431]))
(array([26960.10875267, 36623.54962141]), array([26884.40329585, 36717.94074286]))
(array([26999.0518066 , 36658.06064438]), array([26913.38562885, 36743.51417555]))
(array([26983.07045909, 36643.04077314]), array([26897.40428135, 36728.49430431]))
(array([26960.10875267, 36623.54962141]), array([26884.40329585, 36717.94074286]))
(array([26999.0518066 , 36658.06064438]), array([26913.38562885, 36743.51417555]))
(array([26983.07045909, 36643.04077314]), array([26897.40428135, 36728.49430431]))
(array([26960.10875267, 36623.54962141]), array([26884.40329585, 36717.94074286]))
(array([26005.89422965, 36474.73806953]), array([25891.15118844, 36436.33189173]))
(array([26013.86162665, 36444.4811934 ]), array([25895.24094528, 36420.6037155 ]))
(array([26018.95138272, 36428.75425984]), array

/gpfs/home2/cbisot/pycode/AMFtrack/amftrack/pipeline/functions/image_processing/experiment_class_surf.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.folders["datetime"] = pd.to_datetime(


2023-09-02 13:43:00
2023-09-02 19:43:00
(array([25519.06726077, 36944.58309953]), array([25484.28603711, 36828.68975856]))
(array([25486.57585275, 36951.5353213 ]), array([25466.43801377, 36832.22284316]))
(array([25454.16292509, 36836.67948713]), array([25450.51059875, 36957.62435272]))
(array([25763.39316255, 36976.17701485]), array([25687.90438213, 37070.74151152]))
(array([25761.96389177, 36976.40698459]), array([25656.37541873, 37035.50076058]))
(array([25726.95600862, 36940.21822921]), array([25648.55036234, 37032.37871223]))
(array([25763.39316255, 36976.17701485]), array([25687.90438213, 37070.74151152]))
(array([25761.96389177, 36976.40698459]), array([25656.37541873, 37035.50076058]))
(array([25726.95600862, 36940.21822921]), array([25648.55036234, 37032.37871223]))
(array([25763.39316255, 36976.17701485]), array([25687.90438213, 37070.74151152]))
(array([25761.96389177, 36976.40698459]), array([25656.37541873, 37035.50076058]))
(array([25726.95600862, 36940.21822921]), array

/gpfs/home2/cbisot/pycode/AMFtrack/amftrack/pipeline/functions/image_processing/experiment_class_surf.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.folders["datetime"] = pd.to_datetime(


2023-09-03 11:43:00
2023-09-03 15:47:00
(array([26082.79782359, 36555.8410085 ]), array([26009.7589158, 36652.3102676]))
(array([26075.32500623, 36545.42699998]), array([25989.377363  , 36630.59743278]))
(array([26059.38875797, 36531.35447387]), array([25973.44111473, 36616.52490667]))
(array([26082.79782359, 36555.8410085 ]), array([26009.7589158, 36652.3102676]))
(array([26075.32500623, 36545.42699998]), array([25989.377363  , 36630.59743278]))
(array([26059.38875797, 36531.35447387]), array([25973.44111473, 36616.52490667]))
(array([26082.79782359, 36555.8410085 ]), array([26009.7589158, 36652.3102676]))
(array([26075.32500623, 36545.42699998]), array([25989.377363  , 36630.59743278]))
(array([26059.38875797, 36531.35447387]), array([25973.44111473, 36616.52490667]))
(array([24697.02928227, 34960.53052152]), array([24623.99037449, 35056.99978061]))
(array([24702.83142306, 34962.90047622]), array([24594.36107372, 35016.52121209]))
(array([24691.92691657, 34941.85073185]), array([2458

/gpfs/home2/cbisot/pycode/AMFtrack/amftrack/pipeline/functions/image_processing/experiment_class_surf.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.folders["datetime"] = pd.to_datetime(


2023-09-04 09:42:00
2023-09-04 15:45:00
(array([25403.15229924, 33269.04358766]), array([25332.63528513, 33367.37135953]))
(array([25372.54667216, 33251.9930338 ]), array([25315.83269329, 33358.87860159]))
(array([25355.29500206, 33244.36764364]), array([25287.8836385 , 33344.85002052]))
(array([25460.49223405, 33314.28076938]), array([25340.25794092, 33300.6897887 ]))
(array([25564.98805415, 33853.64135128]), array([25446.84502638, 33879.77992409]))
(array([25558.91552055, 33819.72802683]), array([25446.29485219, 33863.97263167]))
(array([25558.36534636, 33803.92073441]), array([25440.22231859, 33830.05930722]))
(array([25501.80814184, 33267.39805558]), array([25401.32576496, 33199.98669202]))
(array([25479.29728046, 33289.09128425]), array([25389.35820882, 33208.147085  ]))
(array([25450.89946782, 33313.71641384]), array([25383.27760395, 33213.37557706]))
(array([25277.71304258, 32458.63778464]), array([25210.09117871, 32358.29694786]))
(array([25287.29717424, 32445.36911125]), array

/gpfs/home2/cbisot/pycode/AMFtrack/amftrack/pipeline/functions/image_processing/experiment_class_surf.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.folders["datetime"] = pd.to_datetime(


2023-09-05 13:45:00
2023-09-05 19:51:00
(array([25111.81877788, 32750.03797278]), array([25059.6697072 , 32859.22347744]))
(array([25101.44735538, 32744.09463035]), array([25027.57832888, 32839.92973313]))
(array([25079.23418173, 32732.49398742]), array([25005.36515523, 32828.3290902 ]))
(array([25179.09773652, 32785.88750575]), array([25059.4042194 , 32768.15443048]))
(array([25312.81034352, 33719.6783937 ]), array([25191.83008589, 33721.86408901]))
(array([25309.35924306, 33694.73666349]), array([25188.37898543, 33696.9223588 ]))
(array([25307.92587988, 33670.75864293]), array([25186.94562225, 33672.94433824]))
(array([25176.76599883, 32721.9730583 ]), array([25055.7857412 , 32724.15875361]))
(array([25175.31457205, 32696.9952009 ]), array([25054.33431442, 32699.18089621]))
(array([25173.86314527, 32672.0173435 ]), array([25052.88288764, 32674.20303881]))
(array([25194.1971145 , 32776.34389319]), array([25172.15212609, 32657.36902825]))
(array([25197.9300916 , 32782.93974879]), array

In [8]:
for index, vid_obj in enumerate(data_obj.video_objs):
    if check_hasedges(vid_obj) and vid_obj.dataset["mode"] == "BF":
        for edge in vid_obj.edge_objs:
            if "network_begin" in edge.mean_data.keys():
                    break

In [12]:
exp.nx_graph[0][edge_begin][edge_end]

{'weight': 194,
 'pixel_list': [(17294, 6185),
  (17295, 6185),
  (17296, 6186),
  (17297, 6186),
  (17298, 6186),
  (17299, 6186),
  (17300, 6186),
  (17301, 6186),
  (17302, 6187),
  (17303, 6187),
  (17304, 6187),
  (17305, 6187),
  (17306, 6187),
  (17307, 6187),
  (17308, 6188),
  (17309, 6188),
  (17309, 6189),
  (17310, 6190),
  (17311, 6191),
  (17312, 6192),
  (17312, 6193),
  (17313, 6194),
  (17314, 6194),
  (17314, 6195),
  (17315, 6195),
  (17315, 6196),
  (17316, 6197),
  (17317, 6197),
  (17317, 6198),
  (17318, 6198),
  (17319, 6198),
  (17319, 6199),
  (17320, 6199),
  (17321, 6199),
  (17322, 6200),
  (17323, 6201),
  (17324, 6201),
  (17324, 6202),
  (17325, 6202),
  (17326, 6202),
  (17326, 6203),
  (17327, 6203),
  (17328, 6204),
  (17329, 6204),
  (17330, 6205),
  (17331, 6205),
  (17332, 6206),
  (17333, 6206),
  (17334, 6207),
  (17335, 6207),
  (17336, 6208),
  (17337, 6208),
  (17338, 6209),
  (17339, 6209),
  (17340, 6210),
  (17341, 6210),
  (17342, 6211),
 

In [16]:
exp.nx_graph[0].get_edge_data(edge_begin, edge_end)

{'weight': 194,
 'pixel_list': [(17294, 6185),
  (17295, 6185),
  (17296, 6186),
  (17297, 6186),
  (17298, 6186),
  (17299, 6186),
  (17300, 6186),
  (17301, 6186),
  (17302, 6187),
  (17303, 6187),
  (17304, 6187),
  (17305, 6187),
  (17306, 6187),
  (17307, 6187),
  (17308, 6188),
  (17309, 6188),
  (17309, 6189),
  (17310, 6190),
  (17311, 6191),
  (17312, 6192),
  (17312, 6193),
  (17313, 6194),
  (17314, 6194),
  (17314, 6195),
  (17315, 6195),
  (17315, 6196),
  (17316, 6197),
  (17317, 6197),
  (17317, 6198),
  (17318, 6198),
  (17319, 6198),
  (17319, 6199),
  (17320, 6199),
  (17321, 6199),
  (17322, 6200),
  (17323, 6201),
  (17324, 6201),
  (17324, 6202),
  (17325, 6202),
  (17326, 6202),
  (17326, 6203),
  (17327, 6203),
  (17328, 6204),
  (17329, 6204),
  (17330, 6205),
  (17331, 6205),
  (17332, 6206),
  (17333, 6206),
  (17334, 6207),
  (17335, 6207),
  (17336, 6208),
  (17337, 6208),
  (17338, 6209),
  (17339, 6209),
  (17340, 6210),
  (17341, 6210),
  (17342, 6211),
 

In [11]:
edge_begin = int(edge.mean_data["network_begin"])
edge_end = int(edge.mean_data["network_end"])
network_edge = Edge(Node(edge_begin, exp), Node(edge_end, exp), exp)
(
    profiles,
    transects,
    new_section_coord_list,
) = extract_section_profiles_for_edge_exp(
    exp,
    t,
    network_edge,
    resolution=5,
    offset=10,
    step=3,)
                       

TypeError: 'NoneType' object is not subscriptable

In [ ]:
folders = folders.sort_values(by="datetime")

exp = Experiment(directory_targ)
i = indexes[plate_id_video]
exp.load(folders.iloc[i : i + 1], suffix="_labeled")
for t in range(exp.ts):
    exp.load_tile_information(t)

In [ ]:
mpl.rcParams["figure.dpi"] = 50

from random import choice

vid_obj = data_obj.video_objs[21]
vid_obj.plot_speed_arrows(plot_both=True, video_txt_size=30)

In [ ]:
from PIL import Image, ImageDraw, ImageFont

font_size = 24  # Change this to your desired font size
font = ImageFont.truetype(
    "/usr/share/fonts/dejavu/DejaVuSansMono.ttf", font_size
)  # 'arial.ttf' might need to be adjusted based on your system

In [ ]:
def

In [ ]:
weight = 0.1
for vid_obj in data_obj.video_objs:
    if vid_obj.dataset["mode"] == "BF":

        first_frame = vid_obj.get_first_frame()

        first_frame_img = Image.fromarray(first_frame)
        draw = ImageDraw.Draw(first_frame_img)

        # Optionally, specify a font for the text
        # font = ImageFont.truetype("path_to_font.ttf", font_size)

        for edge in vid_obj.edge_objs:
            begin = edge.edge_name.split(",")[0][1:]
            end = edge.edge_name.split(",")[1][1:-1]
            ypos_1 = edge.edge_infos["edge_ypos_1"]
            xpos_1 = edge.edge_infos["edge_xpos_1"]
            ypos_2 = edge.edge_infos["edge_ypos_2"]
            xpos_2 = edge.edge_infos["edge_xpos_2"]

            # Calculate positions for the start and end texts
            start_pos = (0.9 * ypos_1 + 0.1 * ypos_2, 0.9 * xpos_1 + 0.1 * xpos_2)
            end_pos = (0.9 * ypos_2 + 0.1 * ypos_1, 0.9 * xpos_2 + 0.1 * xpos_1)

            # Draw the start and end texts on the image
            draw.text(
                start_pos, str(begin), fill="white", font=font
            )  # , font=font) if using a custom font
            draw.text(
                end_pos, str(end), fill="white", font=font
            )  # , font=font) if using a custom font

        # Save the modified image or display it
        unique_id = vid_obj.dataset["unique_id"]
        path = f"/scratch-shared/amftrack/ML_dataset/{unique_id}.png"
        first_frame_img.save(path)

In [ ]:
load_graphs(exp, directory_targ)

In [ ]:
from amftrack.pipeline.functions.image_processing.extract_width_fun import (
    extract_section_profiles_for_edge_exp,
)

for index, vid_obj in enumerate(data_obj.video_objs):
    if check_hasedges(vid_obj) and vid_obj.dataset["mode"] == "BF":
        for edge in vid_obj.edge_objs:
            edge_begin = int(edge.mean_data["network_begin"])
            edge_end = int(edge.mean_data["network_end"])
            network_edge = Edge(Node(edge_begin, exp), Node(edge_end, exp), exp)
            (
                profiles,
                transects,
                new_section_coord_list,
            ) = extract_section_profiles_for_edge_exp(
                exp,
                t,
                network_edge,
                resolution=5,
                offset=10,
                step=3,
                target_length=120,
            )
            unique_id = vid_obj.dataset["unique_id"]

            os.makedirs(
                f"/scratch-shared/amftrack/ML_dataset/{unique_id}/", exist_ok=True
            )
            for index in [5, 10, 15]:
                if index < len(profiles):
                    path = f"/scratch-shared/amftrack/ML_dataset/{unique_id}/{edge.edge_name}_{index}.npy"
                    np.save(path, profiles[index])

In [ ]:
from amftrack.util.dbx import upload_folder

path = f"/scratch-shared/amftrack/ML_dataset/"
target_drop = "/DATA/CocoTransport/NewMLDataset"
upload_folder(path, target_drop, delete=False)

In [ ]:
ezzz

In [ ]:
edge = vid_obj.edge_objs[0]

In [ ]:
edge_begin = int(edge.mean_data["network_begin"])
edge_end = int(edge.mean_data["network_end"])
edge_begin

In [ ]:
network_edge = Edge(Node(edge_begin, exp), Node(edge_end, exp), exp)

In [ ]:
from amftrack.pipeline.functions.transport_processing.high_mag_videos.plotting import *

plot_full_video(
    exp,
    t,
    downsizing=downsizing,
    points=positions_list,
    video_num=[0],
    edges=[network_edge],
    dilation=10,
    region=region,
    segments=segments,
    nodes=[network_edge.begin, network_edge.end],
    node_size=20,
)

In [ ]:
from amftrack.pipeline.functions.image_processing.extract_width_fun import (
    extract_section_profiles_for_edge_exp,
)

profiles, transects, new_section_coord_list = extract_section_profiles_for_edge_exp(
    exp,
    t,
    network_edge,
    resolution=5,
    offset=10,
    step=3,
    target_length=120,
)

In [ ]:
index = 5
os.mkdir(f"/scratch-shared/amftrack/ML_dataset/{unique_id}/")
path = f"/scratch-shared/amftrack/ML_dataset/{unique_id}/{edge.edge_name}_{index}.npy"
np.save(path, profiles[index])

In [ ]:
edge.edge_name

In [ ]:
fig, ax = plt.subplots()

ax.plot(profiles[10])

In [ ]:
from amftrack.pipeline.functions.transport_processing.high_mag_videos.plotting import *

plot_full_video(
    exp,
    t,
    downsizing=downsizing,
    points=positions_list,
    video_num=[0],
    edges=[network_edge],
    dilation=10,
    region=region,
    # segments = transects[-5:],
    nodes=[network_edge.begin, network_edge.end],
    node_size=10,
)

In [ ]:
px, py = zip(*transects)  # Unpacking points into x and y coordinates

In [ ]:
network_edge.begin.pos(t), network_edge.end.pos(t)

In [ ]:
fig, ax = plt.subplots()
for segment in [transects[0], transects[-1]]:
    x_values = [
        segment[0][0],
        segment[1][0],
    ]  # Extracting x-coordinates of the segment's endpoints
    y_values = [
        segment[0][1],
        segment[1][1],
    ]  # Extracting y-coordinates of the segment's endpoints
    ax.plot(x_values, y_values, marker="o")

In [ ]:
positions = np.array(vid_obj.dataset[["xpos_network", "ypos_network"]])
positions_list = [positions.tolist()]
window = np.array([400, 400])
begin = (positions - window).astype(int)
end = (positions + window).astype(int)
region = [[begin[0], begin[1]], [end[0], end[1]]]
# region = [[100, 100], [2000,2000]]

In [ ]:
shiftx = vid_obj.img_dim[0] * vid_obj.space_res / 1.725 / 2
shifty = vid_obj.img_dim[1] * vid_obj.space_res / 1.725 / 2

In [ ]:
segments = []
for i in range(len(vid_obj.edge_objs)):
    edge = vid_obj.edge_objs[i]
    edge_dat_adr = (
        Path(f"{analysis_folder_root}{edge.mean_data['folder']}") / f"edges_data.csv"
    )
    edges = pd.read_csv(edge_dat_adr)
    edge.edge_infos = edges[edges["edge_name"] == edge.edge_name].iloc[0]
    edge.edge_infos["edge_xpos_1"]
    x_pos_video = edge.mean_data["xpos_network"]
    y_pos_video = edge.mean_data["ypos_network"]

    x_pos1 = (
        edge.edge_infos["edge_xpos_1"] * edge.space_res / 1.725 + x_pos_video - shiftx
    )
    x_pos2 = (
        edge.edge_infos["edge_xpos_2"] * edge.space_res / 1.725 + x_pos_video - shiftx
    )
    y_pos1 = (
        edge.edge_infos["edge_ypos_1"] * edge.space_res / 1.725 + y_pos_video - shifty
    )
    y_pos2 = (
        edge.edge_infos["edge_ypos_2"] * edge.space_res / 1.725 + y_pos_video - shifty
    )
    dist = np.linalg.norm(np.array([x_pos1, y_pos1]) - np.array([x_pos2, y_pos2]))
    if dist > 40:
        segments.append([[x_pos1, y_pos1], [x_pos2, y_pos2]])
        # print(dist)

In [ ]:
edges = get_all_edges(exp, t)

edges = [edge for edge in edges if dist_edge(edge, positions, t) <= 100]
pixels = [pixel for edge in edges for pixel in edge.pixel_list(t)]
pixels = [pixel for pixel in pixels if np.linalg.norm(pixel - positions) <= 100]

In [ ]:
segment_points = []
for begin, end in segments:
    # Include the start point, interpolated points, and the end point
    interpolated_points = interpolate_points(begin, end)
    segment_points.extend(interpolated_points)
    segment_points.append(end)  # Ensure the end point is included

segment_points = np.array(segment_points)

In [ ]:
positions_list = [positions.tolist()]
plt.close("all")
t = 0
vmax = 9
vmin = 3
nodes = get_all_nodes(exp, t)
edges = get_all_edges(exp, t)
edges = [edge for edge in edges if dist_edge(edge, positions, t) <= 150]
downsizing = 1

plot_full_video(
    exp,
    t,
    downsizing=downsizing,
    points=positions_list,
    video_num=[0],
    edges=edges,
    dilation=5,
    region=region,
    segments=segments,
)

In [ ]:
fig, ax = plt.subplots()

# Plotting point cloud
px, py = zip(*pixels)  # Unpacking points into x and y coordinates
ax.scatter(px, py, color="blue")  # Plot points in blue
px, py = zip(*segment_points)  # Unpacking points into x and y coordinates
ax.scatter(px, py, color="red")  # Plot points in blue

In [ ]:
Rfound, tfound = register_rot_trans(vid_obj, exp, t, dist=100)

In [ ]:
transformed_points = np.transpose(
    Rfound @ np.transpose(segment_points)[[0, 1], :] + tfound[:, np.newaxis]
)

In [ ]:
fig, ax = plt.subplots()

# Plotting point cloud
px, py = zip(*pixels)  # Unpacking points into x and y coordinates
ax.scatter(px, py, color="blue")  # Plot points in
px, py = zip(*transformed_points)  # Unpacking points into x and y coordinates
ax.scatter(px, py, color="red")  # Plot points in blue

In [ ]:
edges = get_all_edges(exp, t)

edges = [
    edge
    for edge in edges
    if dist_edge(edge, transform(positions, Rfound, tfound), t) <= 100
]
network_edge_segments = [edge.pixel_list(t) for edge in edges]
network_edge_segments = [
    [pixel for pixel in pixels if np.linalg.norm(pixel - positions) <= 1.5 * dist]
    for pixels in network_edge_segments
]
network_edge_segments = [[pixels[0], pixels[-1]] for pixels in network_edge_segments]
# edge_names = [edge.begin. for edge in edges]

In [ ]:
def initialize_transformation():
    return np.array([[1, 0], [0, 1]]), np.array([0, 0])

In [ ]:
Rcurrent, tcurrent = initialize_transformation()
mapping, dist, Rfound, tfound = make_whole_mapping(
    vid_obj, exp, t, dist=100, R=Rcurrent, trans=tcurrent
)

In [ ]:
mapping

In [ ]:
mpl.rcParams["figure.dpi"] = 100

fig, ax = plt.subplots()

for segment in network_edge_segments:
    x_values = [
        segment[0][0],
        segment[1][0],
    ]  # Extracting x-coordinates of the segment's endpoints
    y_values = [
        segment[0][1],
        segment[1][1],
    ]  # Extracting y-coordinates of the segment's endpoints
    ax.plot(
        x_values, y_values, marker="o"
    )  # Plotting the segment with points at the endpoints

px, py = zip(*transformed_points)  # Unpacking points into x and y coordinates
ax.scatter(px, py, color="red")  # Plot points in blue

In [ ]:
edge_names = [edge.edge_name for edge in vid_obj.edge_objs]
segments_final = get_segments_ends(vid_obj, shiftx, shifty, 0, Rfound, tfound)

In [ ]:
mpl.rcParams["figure.dpi"] = 100

fig, ax = plt.subplots()

for segment in network_edge_segments:
    x_values = [
        segment[0][0],
        segment[1][0],
    ]  # Extracting x-coordinates of the segment's endpoints
    y_values = [
        segment[0][1],
        segment[1][1],
    ]  # Extracting y-coordinates of the segment's endpoints
    ax.plot(
        x_values, y_values, marker="o", color="blue"
    )  # Plotting the segment with points at the endpoints
for segment in segments_final:
    x_values = [
        segment[0][0],
        segment[1][0],
    ]  # Extracting x-coordinates of the segment's endpoints
    y_values = [
        segment[0][1],
        segment[1][1],
    ]  # Extracting y-coordinates of the segment's endpoints
    ax.plot(
        x_values, y_values, marker="o", color="red"
    )  # Plotting the segment with points at the endpoints

In [ ]:
def

In [ ]:
for transport_edge_name, transport_edge_segment in zip(edge_names, segments_final):
    index = edges.index(mapping[transport_edge_name])
    segment_network = np.array(network_edge_segments[index])
    vector_network = segment_network[0] - segment_network[1]
    segment_video = np.array(transport_edge_segment)
    vector_video = segment_video[0] - segment_video[1]
    print(np.dot(vector_network, vector_video))

In [ ]:
index = edges.index(mapping[transport_edge_name])
segment_network = np.array(network_edge_segments[index])
vector_network = segment_network[0] - segment_network[1]
segment_video = np.array(transport_edge_segment)
vector_video = segment_video[0] - segment_video[1]
np.dot(vector_network, vector_video)

In [ ]:
vector_video